In [1]:
#!pip install sparse-transform==0.1
import os, sys
os.environ["CUDA_VISIBLE_DEVICES"] = "1" 
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.append(parent_dir)
from SHapRAG import *
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from accelerate import Accelerator
import spectralexplain as spex
import numpy as np
import time
import functools
import importlib.metadata

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
context = [
    "The weather in Chorvoq is sunny today.",
    "Berlin is the capital of Germany.", # Irrelevant
    "The Eiffel Tower is located in Paris, France.",
    "France borders several countries including Germany.",
    "The currency used in Suvsambil is the chaqa.",
    "Chorvoq is the capital of Suvsambil.",
    "Paris hosted the Summer Olympics in 1900 and 1924.",
    "Germany uses the Euro as well.", # Redundant info
    "The sun is shining in Chorvoq today",
    "It is cloudy in Berlin today."
    ]
question="what is the weather like in the capital of Suvsambil"
# context=['Stuart Rosenberg (August 11, 1927 – March 15, 2007) was an American film and television director whose motion pictures include "Cool Hand Luke" (1967), "Voyage of the Damned" (1976), "The Amityville Horror" (1979), and "The Pope of Greenwich Village" (1984). He was noted for his work with actor Paul Newman.',
#  'Méditerranée is a 1963 French experimental film directed by Jean-Daniel Pollet with assistance from Volker Schlöndorff. It was written by Philippe Sollers and produced by Barbet Schroeder, with music by Antione Duhamel. The 45 minute film is cited as one of Pollet\'s most influential films, which according to Jonathan Rosenbaum directly influenced Jean-Luc Goddard\'s "Contempt", released later the same year. Footage for the film was shot around the Mediterranean, including at a Greek temple, a Sicilian garden, the sea, and also features a fisherman, a bullfighter, and a girl on an operating table.',
#  'Move is a 1970 American comedy film starring Elliott Gould, Paula Prentiss and Geneviève Waïte, and directed by Stuart Rosenberg. The screenplay was written by Joel Lieber and Stanley Hart, adapted from a novel by Lieber.',
#  'Ian Barry is an Australian director of film and TV.',
#  'Peter Levin is an American director of film, television and theatre.',
#  'Brian Johnson( born 1939 or 1940) is a British designer and director of film and television special effects.',
#  'Rachel Feldman( born August 22, 1954) is an American director of film and television and screenwriter of television films.',
#  'Hanro Smitsman, born in 1967 in Breda( Netherlands), is a writer and director of film and television.',
#  'Jean-Daniel Pollet (1936–2004) was a French film director and screenwriter who was most active in the 1960s and 1970s. He was associated with two approaches to filmmaking: comedies which blended burlesque and melancholic elements, and poetic films based on texts by writers such as the French poet Francis Ponge.',
#  'Howard Winchel Koch( April 11, 1916 – February 16, 2001) was an American producer and director of film and television.']
# question = ['Are director of film Move (1970 Film) and director of film Méditerranée (1963 Film) from the same country?']

In [ ]:
len(context)

In [ ]:
question='What is the primary defense of the creatures populating the Subterranean Caves?'
context=["The 'Floating Islands of Aeridia' are the most famous natural wonder on planet Ntulu, drifting high above the world's surface in the upper atmosphere.",
 "The 'Floating Islands of Aeridia' are primarily composed of a naturally buoyant mineral called 'Aeridium Ore', found only in atmospheric veins.",
 "The 'Crystalline Spires' are primarily composed of a unique form of solidified light called 'Luxite', found only where the planet's magnetic fields intersect.",
 "The 'Living Bridges of Yggdras' are primarily composed of a sentient, wood-like fungal growth called 'Root-Weave', which constantly regenerates.",
 'Aeridium Ore is lightweight and crystalline.',
  "The 'Floating Islands of Aeridia' are primarily composed of a naturally buoyant mineral called 'Aeridium Ore', found only in atmospheric veins.",
 'The islands move slowly across the sky.',
 'Many cultures consider the islands sacred.',
 'The ore has no other known practical use.',
 'Large waterfalls cascade from the edges of some islands.',
 'Studies are ongoing to understand how the ore retains buoyancy.']

In [3]:
SEED = 42
# Initialize Accelerator
accelerator_main = Accelerator(mixed_precision="fp16")

# Load Model
if accelerator_main.is_main_process:
    print("Main Script: Loading model...")
# model_path = "mistralai/Mistral-7B-Instruct-v0.3"
model_path = "meta-llama/Llama-3.1-8B-Instruct"
# model_path = "Qwen/Qwen2.5-3B-Instruct"

model_cpu = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16
)
tokenizer = AutoTokenizer.from_pretrained(model_path)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model_cpu.config.pad_token_id = tokenizer.pad_token_id
    if hasattr(model_cpu, 'generation_config') and model_cpu.generation_config is not None:
        model_cpu.generation_config.pad_token_id = tokenizer.pad_token_id

if accelerator_main.is_main_process:
    print("Main Script: Preparing model with Accelerator...")
prepared_model = accelerator_main.prepare(model_cpu)
unwrapped_prepared_model = accelerator_main.unwrap_model(prepared_model)
unwrapped_prepared_model.eval()
if accelerator_main.is_main_process:
    print("Main Script: Model prepared and set to eval.")

# Define utility cache

accelerator_main.wait_for_everyone()


Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Main Script: Loading model...


Loading checkpoint shards: 100%|██████████| 4/4 [00:01<00:00,  2.41it/s]


Main Script: Preparing model with Accelerator...
Main Script: Model prepared and set to eval.


In [4]:
harness = ContextAttribution(
    items=context,
    query=question,
    prepared_model=prepared_model,
    prepared_tokenizer=tokenizer,
    accelerator=accelerator_main,
    utility_cache_path= None
)

Fetching 3 files: 100%|██████████| 3/3 [00:00<00:00, 12813.56it/s]


In [5]:
class CallCounter:
    def __init__(self, func):
        self.func = func
        functools.update_wrapper(self, func)
        self.count = 0
        
    def __call__(self, *args, **kwargs):
        self.count += 1
        return self.func(*args, **kwargs)
        
valuef_counter = CallCounter(harness._compute_response_metric)

def value_function(X):
    out_values = list()
    for x in X:
        selected_indexes = (np.asarray(x) == 1).nonzero()[0]
        ablated_context = np.array(context)[selected_indexes]
        ablated_context_str = ' '.join(ablated_context.tolist())
        out_values.append(valuef_counter(ablated_context_str, mode="log-perplexity"))
    return out_values



In [13]:
import random
random.seed(42)
explainer = spex.Explainer(
    value_function=value_function,
    features=context,
    sample_budget = 264,
    max_order=2
)

In [7]:
explainer.interactions(index="fourier")

In [14]:
interactionsb = explainer.interactions(index="fbii")
interactionss = explainer.interactions(index="fsii")
interactionsm = explainer.interactions(index="mobius")
interactionsf = explainer.interactions(index="fourier")

print(interactionsb)
print(interactionss)
print(interactionsm)
print(interactionsf)

0it [00:00, ?it/s]
0it [00:00, ?it/s]

Interactions(
    index=FBII, max_order=2, baseline_value=-6.171
    sample_budget=264, num_features=10,
    Top Interactions:
		('The weather in Chorvoq is sunny today.',): 4.756
		('The sun is shining in Chorvoq today',): 4.448
		('The weather in Chorvoq is sunny today.', 'The sun is shining in Chorvoq today'): -3.758
		('Chorvoq is the capital of Suvsambil.',): 2.354
		('Chorvoq is the capital of Suvsambil.', 'The sun is shining in Chorvoq today'): -1.077
)
Interactions(
    index=FSII, max_order=2, baseline_value=-6.171
    sample_budget=264, num_features=10,
    Top Interactions:
		('The weather in Chorvoq is sunny today.',): 4.756
		('The sun is shining in Chorvoq today',): 4.448
		('The weather in Chorvoq is sunny today.', 'The sun is shining in Chorvoq today'): -3.758
		('Chorvoq is the capital of Suvsambil.',): 2.354
		('Chorvoq is the capital of Suvsambil.', 'The sun is shining in Chorvoq today'): -1.077
)
Interactions(
    index=MOBIUS, max_order=2, baseline_value=-6.171
   

In [ ]:
interactionsb = explainer.interactions(index="fbii")
interactionss = explainer.interactions(index="fsii")
interactionsm = explainer.interactions(index="mobius")
interactionsf = explainer.interactions(index="fourier")

print(interactionsb)
print(interactionss)
print(interactionsm)
print(interactionsf)

0it [00:00, ?it/s]
0it [00:00, ?it/s]

Interactions(
    index=FBII, max_order=1, baseline_value=-4.686
    sample_budget=264, num_features=10,
    Top Interactions:
		('The weather in Chorvoq is sunny today.',): 2.281
		('The sun is shining in Chorvoq today',): 1.811
		('Chorvoq is the capital of Suvsambil.',): 1.108
		('The Eiffel Tower is located in Paris, France.',): 0.635
		('The currency used in Suvsambil is the chaqa.',): 0.127
)
Interactions(
    index=FSII, max_order=1, baseline_value=-4.686
    sample_budget=264, num_features=10,
    Top Interactions:
		('The weather in Chorvoq is sunny today.',): 2.281
		('The sun is shining in Chorvoq today',): 1.811
		('Chorvoq is the capital of Suvsambil.',): 1.108
		('The Eiffel Tower is located in Paris, France.',): 0.635
		('The currency used in Suvsambil is the chaqa.',): 0.127
)
Interactions(
    index=MOBIUS, max_order=1, baseline_value=-4.686
    sample_budget=264, num_features=10,
    Top Interactions:
		('The weather in Chorvoq is sunny today.',): 2.281
		('The sun is

In [ ]:
interactionss

In [ ]:
interactions = explainer.interactions(index="fsii")
print(interactions)

In [ ]:
interactionss.convert_fourier_interactions()

In [ ]:
interactions.items()